In [14]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
from sklearn.compose import make_column_transformer 
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
df=pd.read_csv("EDA.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   company     704 non-null    object
 1   year        704 non-null    int64 
 2   Price       704 non-null    int64 
 3   fuel_type   704 non-null    object
 4   Driven      704 non-null    int64 
 5   Car_Name    704 non-null    object
 6   Model_Info  704 non-null    object
dtypes: int64(3), object(4)
memory usage: 38.6+ KB


In [4]:
df.columns

Index(['company', 'year', 'Price', 'fuel_type', 'Driven', 'Car_Name',
       'Model_Info'],
      dtype='object')

In [5]:
X=df[['company', 'year', 'fuel_type', 'Driven', 'Car_Name','Model_Info']]
y=df[['Price']]

In [16]:
ohe.categories_

[array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
        'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
        'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
        'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
       dtype=object),
 array(['Diesel', 'LPG', 'Petrol'], dtype=object),
 array(['Audi A3', 'Audi A4', 'Audi A6', 'Audi Q3', 'Audi Q5', 'BMW 3',
        'BMW 5', 'BMW 7', 'BMW X1', 'Chevrolet Beat', 'Chevrolet Cruze',
        'Chevrolet Enjoy', 'Chevrolet Sail', 'Chevrolet Spark',
        'Chevrolet Tavera', 'Datsun GO', 'Datsun Go', 'Datsun Redi',
        'Fiat Linea', 'Fiat Punto', 'Force Motors', 'Ford EcoSport',
        'Ford Endeavor', 'Ford Fiesta', 'Ford Figo', 'Ford Fusion',
        'Hindustan Motors', 'Honda Amaze', 'Honda Brio', 'Honda City',
        'Honda Jazz', 'Honda Mobilio', 'Honda WR', 'Hyundai Accent',
        'Hyundai Creta', 'Hyundai Elantra', 'Hyundai Elite', 'Hyundai Eon',
        'Hyund

In [15]:
#fit OneHotEncoder
ohe=OneHotEncoder()
ohe.fit(X[['company','fuel_type','Car_Name','Model_Info']])


In [ ]:
#using make_column_transformer to encode and for feature scaling
mct=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['company','fuel_type','Car_Name','Model_Info']),
                            (StandardScaler(),['year','Driven']),
                             remainder='passthrough')


In [17]:
# creating pipeline
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
pipe=make_pipeline(mct,lr)

In [24]:
# using for loop in which we testing the model using diff random_state and finding the best one
scores=[]
for i in range(0,101):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    y_pred=pd.DataFrame(data=y_pred,columns=['result'])
    pd.set_option('display.float_format', '{:.0f}'.format)
    result = pd.concat([y_test.reset_index(drop=True), y_pred.reset_index(drop=True)], axis=1)
    result
    score=r2_score(result['Price'],result['result'])
    scores.append(score)
    

In [25]:
# finding index of high score  and printing it
index=np.argmax(scores)
print(np.max(scores))
index

0.9342863272876403


48

In [26]:
# testing model using best random_state  again
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=48)
#creating pipeline again
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Vol...
       'i10 Magna 1.2 Kappa VTVT', 'i10 Magna AT 1.2 Kappa VTVT',
       'i10 Sportz 1.2 Kappa VTVT', 'i10 Sportz O 1.2 Kappa VTVT', 'i20',
       'i20 Asta 1.2', 'i20 Asta 1.4 CRDI', 'i20 Magna 1.2',
       'i20 Sportz 1.2', 'sDrive20d', 'xDrive20d xLine'], dtype=object)]),
                                                  ['company', 'fuel_type',
                                                   'Car_Name', 'Model_Info']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['year', 'Driven'])])),
                ('linearregression', LinearRegression())])

In [27]:
#testing model using myinput
myinput=pd.DataFrame([["Ford",2012,"Diesel",42000,'Ford Figo','Diesel EXI']],columns=['company','year','fuel_type','Driven','Car_Name','Model_Info'])
myinput

,company,year,fuel_type,Driven,Car_Name,Model_Info
0,Ford,2012,Diesel,42000,Ford Figo,Diesel EXI


In [29]:
res=pipe.predict(myinput)
result=np.round(res[0,0])
result

164095.0

In [34]:
import pickle as pkl
# Save your trained model pipeline to a file
with open("Car-price-prediction.pkl", "wb") as f:
    pkl.dump(pipe, f)
